In [14]:
import torchtext
import torch
import numpy as np
import pandas as pd
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader, Dataset
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe
# GloVe is well embedded in the torchtext package, easy to use
glove = GloVe(name='6B', dim=100)


In [2]:
# Get vectors
tensor = glove.get_vecs_by_tokens(['', '1998', '199999998', ',', 'cat'], True)
print(tensor)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

In [3]:
myvocab = glove.itos
print(len(myvocab))
print(myvocab[0], myvocab[1], myvocab[2], myvocab[3])

400000
the , . of


In [4]:
def get_counterpart(x1, y1, x2):
    """Find y2 that makes x1-y1=x2-y2"""
    x1_id = glove.stoi[x1]
    y1_id = glove.stoi[y1]
    x2_id = glove.stoi[x2]
    x1, y1, x2 = glove.get_vecs_by_tokens([x1, y1, x2], True)
    target = x2 - x1 + y1
    max_sim = 0
    max_id = -1
    for i in range(len(myvocab)):
        vector = glove.get_vecs_by_tokens([myvocab[i]], True)[0]
        cossim = torch.dot(target, vector)
        if cossim > max_sim and i not in {x1_id, y1_id, x2_id}:
            max_sim = cossim
            max_id = i
    return myvocab[max_id]


print(get_counterpart('man', 'woman', 'king'))
print(get_counterpart('more', 'less', 'long'))
print(get_counterpart('apple', 'red', 'banana'))

queen
short
yellow


In [6]:
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer('basic_english')
print(tokenizer('a, b'))

['a', ',', 'b']


In [15]:
train = pd.read_csv('/Users/qiaochufeng/Documents/GitHub/assignment-2-text-classification-Carrief-0908/nlp-getting-started/train.csv')
test = pd.read_csv('/Users/qiaochufeng/Documents/GitHub/assignment-2-text-classification-Carrief-0908/nlp-getting-started/test.csv')
submit = pd.read_csv('/Users/qiaochufeng/Documents/GitHub/assignment-2-text-classification-Carrief-0908/nlp-getting-started/sample_submission.csv')

In [12]:
print('Train size:', train.shape)
print('Test size:', test.shape)
train.head()

Train size: (7613, 5)
Test size: (3263, 4)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [24]:
total_lines = train['text'].tolist()
test_lines = test['text'].tolist()
tgt0_lines = train[(train.target==0)]['text'].tolist()
tgt1_lines = train[(train.target==1)]['text'].tolist()

In [37]:
GLOVE_DIM = 100
GLOVE = GloVe(name='6B', dim=GLOVE_DIM)

class twitterDataset_train():
    def __init__(self):
        super().__init__()
        self.tokenizer = get_tokenizer('basic_english')
        pos_lines = train[(train.target==1)]['text'].tolist()
        neg_lines = train[(train.target==0)]['text'].tolist()
        self.lines = pos_lines + neg_lines
        self.pos_length = len(pos_lines)
        self.neg_length = len(neg_lines)

    def __len__(self):
        return self.pos_length + self.neg_length

    def __getitem__(self, index):
        sentence = self.tokenizer(self.lines[index])
        x = GLOVE.get_vecs_by_tokens(sentence)
        label = 1 if index < self.pos_length else 0
        return x, label
    
class twitterDataset_test():
    def __init__(self):
        super().__init__()
        self.tokenizer = get_tokenizer('basic_english')
        self.lines = test['text'].tolist()

    def __len__(self):
        return self.lines

def collate_fn(batch):
    x, y = zip(*batch)
    x_pad = pad_sequence(x, batch_first=True)
    y = torch.Tensor(y)
    return x_pad, y



In [38]:
import torch
from torch.nn.utils.rnn import pad_sequence

def get_dataloader():
    def collate_fn(batch):
        x, y = zip(*batch)
        x_pad = pad_sequence(x, batch_first=True)
        y = torch.Tensor(y)
        return x_pad, y

    train_dataloader = DataLoader(twitterDataset_train,
                    batch_size=32,
                    shuffle=True,
                    collate_fn=collate_fn)
    test_dataloader = DataLoader(twitterDataset_test,
                    batch_size=32,
                    shuffle=True,
                    collate_fn=collate_fn)
    return train_dataloader, test_dataloader

In [39]:
from torch import nn
class RNN(torch.nn.Module):
    def __init__(self, hidden_units=64, dropout_rate=0.5):
        super().__init__()
        self.drop = nn.Dropout(dropout_rate)
        self.rnn = nn.GRU(GLOVE_DIM, hidden_units, 1, batch_first=True)
        self.linear = nn.Linear(hidden_units, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x: torch.Tensor):
        # x shape: [batch, max_word_length, embedding_length]
        emb = self.drop(x)
        output, _ = self.rnn(emb)
        output = output[:, -1]
        output = self.linear(output)
        output = self.sigmoid(output)

        return output

device = 'cpu'
train_dataloader, test_dataloader = get_dataloader()
model = RNN().to(device)

TypeError: object of type 'type' has no len()

In [ ]:
# 交上去的记录 baseline
# Epoch 0. loss: 0.6509764790534973
# Epoch 1. loss: 0.5327534079551697
# Epoch 2. loss: 0.4986315965652466
# Epoch 3. loss: 0.47606101632118225
# Epoch 4. loss: 0.4756905734539032
# Epoch 5. loss: 0.4689613878726959
# Epoch 6. loss: 0.46437057852745056
# Epoch 7. loss: 0.455255925655365
# Epoch 8. loss: 0.457621693611145
# Epoch 9. loss: 0.45233598351478577
# Epoch 10. loss: 0.4378981590270996
# Epoch 11. loss: 0.44445234537124634
# Epoch 12. loss: 0.43523845076560974
# Epoch 13. loss: 0.42682886123657227
# Epoch 14. loss: 0.4274866580963135
# Epoch 15. loss: 0.41839489340782166
# Epoch 16. loss: 0.4149070382118225
# Epoch 17. loss: 0.41745632886886597
# Epoch 18. loss: 0.4122700095176697
# Epoch 19. loss: 0.41320765018463135
# Epoch 20. loss: 0.4069027602672577
# Epoch 21. loss: 0.3929371237754822
# Epoch 22. loss: 0.40475574135780334
# Epoch 23. loss: 0.39060503244400024
# Epoch 24. loss: 0.388701468706131
# Epoch 25. loss: 0.3816075325012207
# Epoch 26. loss: 0.38171839714050293
# Epoch 27. loss: 0.3800078332424164
# Epoch 28. loss: 0.3772260248661041
# Epoch 29. loss: 0.3746449649333954
# Epoch 30. loss: 0.3681471645832062
# Epoch 31. loss: 0.3671283721923828
# Epoch 32. loss: 0.3662094175815582
# Epoch 33. loss: 0.3621152341365814
# Epoch 34. loss: 0.3589635193347931
# Epoch 35. loss: 0.36246246099472046
# Epoch 36. loss: 0.3522266745567322
# Epoch 37. loss: 0.35584887862205505
# Epoch 38. loss: 0.3515584170818329
# Epoch 39. loss: 0.3519766628742218
# Epoch 40. loss: 0.3430260717868805
# Epoch 41. loss: 0.339802086353302
# Epoch 42. loss: 0.33826959133148193
# Epoch 43. loss: 0.34149956703186035
# Epoch 44. loss: 0.3370189964771271
# Epoch 45. loss: 0.324999064207077
# Epoch 46. loss: 0.32812047004699707
# Epoch 47. loss: 0.32268932461738586
# Epoch 48. loss: 0.3205323815345764
# Epoch 49. loss: 0.315046489238739
# Epoch 50. loss: 0.3277100622653961
# Epoch 51. loss: 0.3237587511539459
# Epoch 52. loss: 0.3185259997844696
# Epoch 53. loss: 0.31613194942474365
# Epoch 54. loss: 0.3110210597515106
# Epoch 55. loss: 0.3070056140422821
# Epoch 56. loss: 0.31371772289276123
# Epoch 57. loss: 0.30971306562423706
# Epoch 58. loss: 0.2946414649486542
# Epoch 59. loss: 0.3195744752883911
# Epoch 60. loss: 0.3035644590854645
# Epoch 61. loss: 0.2950517535209656
# Epoch 62. loss: 0.30289679765701294
# Epoch 63. loss: 0.29878732562065125
# Epoch 64. loss: 0.29898324608802795
# Epoch 65. loss: 0.30125465989112854
# Epoch 66. loss: 0.29164087772369385
# Epoch 67. loss: 0.2961573600769043
# Epoch 68. loss: 0.28568440675735474
# Epoch 69. loss: 0.2909999191761017
# Epoch 70. loss: 0.2963806390762329
# Epoch 71. loss: 0.2928559184074402
# Epoch 72. loss: 0.2847306728363037
# Epoch 73. loss: 0.2902323007583618
# Epoch 74. loss: 0.2823370695114136
# Epoch 75. loss: 0.2800539433956146
# Epoch 76. loss: 0.28621771931648254
# Epoch 77. loss: 0.27567166090011597
# Epoch 78. loss: 0.27030372619628906
# Epoch 79. loss: 0.2739236056804657
# Epoch 80. loss: 0.2746753990650177
# Epoch 81. loss: 0.2704813778400421
# Epoch 82. loss: 0.26951634883880615
# Epoch 83. loss: 0.2676199972629547
# Epoch 84. loss: 0.2585124969482422
# Epoch 85. loss: 0.26896658539772034
# Epoch 86. loss: 0.27735984325408936
# Epoch 87. loss: 0.2699267864227295
# Epoch 88. loss: 0.26850542426109314
# Epoch 89. loss: 0.26941996812820435
# Epoch 90. loss: 0.26678943634033203
# Epoch 91. loss: 0.2646208107471466
# Epoch 92. loss: 0.25165173411369324
# Epoch 93. loss: 0.2523617446422577
# Epoch 94. loss: 0.26316702365875244
# Epoch 95. loss: 0.2520638406276703
# Epoch 96. loss: 0.24962987005710602
# Epoch 97. loss: 0.2640274465084076
# Epoch 98. loss: 0.24880437552928925
# Epoch 99. loss: 0.25341278314590454
# Accuracy: 0.7850262697022767
